In [2]:
import codecs
import html
from bs4 import BeautifulSoup
import pandas as pd
from unidecode import unidecode
data_ACM = pd.read_csv(r'C:\Users\eetuj\OneDrive - TUNI.fi\DILA\groupProject\DILA_project\ACM.csv')
data_DBLP = pd.read_csv(r'C:\Users\eetuj\OneDrive - TUNI.fi\DILA\groupProject\DILA_project\DBLP2_new.csv')
s = '&#228;'
s = codecs.decode(s, "unicode_escape")
print(s)
#data_ACM['venue'] = data_ACM['venue'].to_string()

&#228;


In [3]:
data_ACM['venue'].str.strip()

0          International Conference on Management of Data
1          International Conference on Management of Data
2          International Conference on Management of Data
3          International Conference on Management of Data
4          International Conference on Management of Data
                              ...                        
2289                                Very Large Data Bases
2290    The VLDB Journal &mdash; The International Jou...
2291                                Very Large Data Bases
2292                                    ACM SIGMOD Record
2293                                Very Large Data Bases
Name: venue, Length: 2294, dtype: object

In [4]:
data_ACM.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
data_DBLP.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)

In [5]:
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: BeautifulSoup(str(x), "html.parser"))

In [6]:
#data_ACM['venue'] = data_ACM['venue'].apply(lambda x: BeautifulSoup(str(x), "html.parser"))
data_ACM['venue'] = data_ACM['venue'].apply(lambda x: html.unescape(x) if isinstance(x, str) else x)

In [7]:
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: unidecode(str(x)))
data_DBLP['authors'] = data_DBLP['authors'].apply(lambda x: unidecode(str(x)))

In [8]:
data_ACM['title'] = data_ACM['title'].apply(lambda x: x.upper())
data_ACM['authors'] = data_ACM['authors'].apply(lambda x: x.upper())
#data_ACM['venue'] = data_ACM['venue'].str.upper()

In [9]:
data_DBLP['title'] = data_DBLP['title'].apply(lambda x: x.upper())
data_DBLP['authors'] = data_DBLP['authors'].apply(lambda x: x.upper())
#data_DBLP['venue'] = data_DBLP['venue'].str.upper()

In [10]:
data_ACM.replace("Very Large Data Bases", "VLDB", inplace=True)
data_ACM.replace("International Conference on Management of Data", "SIGMOD Conference", inplace=True)
data_ACM.replace("ACM SIGMOD Record ", "SIGMOD Record", inplace=True)
data_ACM.replace("ACM Transactions on Database Systems (TODS) ", "ACM Trans. Database Syst. ", inplace=True)
data_ACM.replace("The VLDB Journal — The International Journal on Very Large Data Bases", "VLDB J.", inplace=True)

In [11]:
data_ACM.to_csv(r'C:\Users\eetuj\OneDrive - TUNI.fi\DILA\groupProject\DILA_project\ACM3.csv', encoding='utf-8', index=False)
data_DBLP.to_csv(r'C:\Users\eetuj\OneDrive - TUNI.fi\DILA\groupProject\DILA_project\DBLP3.csv', encoding='utf-8', index=False)

In [12]:
print(data_ACM.head(10))
print(data_DBLP.head(10))

       id                                              title  \
0  304586  THE WASA2 OBJECT-ORIENTED WORKFLOW MANAGEMENT ...   
1  304587  A USER-CENTERED INTERFACE FOR QUERYING DISTRIB...   
2  304589  WORLD WIDE DATABASE-INTEGRATING THE WEB, CORBA...   
3  304590           XML-BASED INFORMATION MEDIATION WITH MIX   
4  304582  THE CCUBE CONSTRAINT OBJECT-ORIENTED DATABASE ...   
5  304583  THE CORNELL JAGUAR PROJECT: ADDING MOBILITY TO...   
6  304584  THE ACTIVE MULTISYNC CONTROLLER OF THE CUBETRE...   
7  304585                  THE JUNGLE DATABASE SEARCH ENGINE   
8  306112  ADEPT: AN AGENT-BASED APPROACH TO BUSINESS PRO...   
9  306115  A COMPONENTIZED ARCHITECTURE FOR DYNAMIC ELECT...   

                                             authors              venue  year  
0                    GOTTFRIED VOSSEN, MATHIAS WESKE  SIGMOD Conference  1999  
1                  ISABEL F. CRUZ, KIMBERLY M. JAMES  SIGMOD Conference  1999  
2  ATHMAN BOUGUETTAYA, BOUALEM BENATALLAH, LILY H...  S

In [ ]:
#data_ACM['venue'] = data_ACM['venue'].to_string()

In [ ]:
data_ACM.dtypes

In [26]:
#for col in data_ACM.columns:
#   data_ACM[col] = data_ACM[col].apply(lambda x: html.unescape(x) if isinstance(x, str) else x)

# Data preparation:

In [13]:
#data_ACM.title.str.lstrip()
#data_ACM.authors.str.lstrip()
#data_ACM.venue.str.lstrip()
print(data_ACM.loc[2160, 'venue'])
#print(data_ACM.dtypes)

VLDB


In [14]:
print(len(pd.unique(data_ACM['venue'])))
print(len(pd.unique(data_DBLP['venue'])))

5
5


In [15]:
print(data_ACM.isnull().sum())

id         0
title      0
authors    0
venue      0
year       0
dtype: int64


In [16]:
print(data_ACM.authors)

0                         GOTTFRIED VOSSEN, MATHIAS WESKE
1                       ISABEL F. CRUZ, KIMBERLY M. JAMES
2       ATHMAN BOUGUETTAYA, BOUALEM BENATALLAH, LILY H...
3       CHAITAN BARU, AMARNATH GUPTA, BERTRAM LUDASCHE...
4       ALEXANDER BRODSKY, VICTOR E. SEGAL, JIA CHEN, ...
                              ...                        
2289                       ALFONS KEMPER, DONALD KOSSMANN
2290    PHILIP A. BERNSTEIN, YANNIS IOANNIDIS, RAGHU R...
2291                                  RALF HARTMUT GUTING
2292                                ALEXANDER A. ANISIMOV
2293                 JANET L. WIENER, JEFFREY F. NAUGHTON
Name: authors, Length: 2272, dtype: object


In [17]:
print(len(data_ACM))
print(len(data_DBLP))

2272
2576


In [18]:
data_ACM.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
data_DBLP.drop_duplicates(subset=['title', 'authors', 'venue', 'year'],inplace=True)
print(len(data_ACM))
print(len(data_DBLP))

2271
2576


In [17]:
#d2['authors'] = data_ACM.apply(lambda x : x.split(","))
#for i, row in data_ACM.iterrows():
 #   data_ACM.at[i, 'authors'] = data_ACM.at[i, 'authors'].split(",")
#data_ACM['authors'] = data_ACM['authors'].str.split(", ")
#data_ACM['authors'] = data_ACM['authors'].str.split(" ")

In [18]:
data_ACM.head()
#html_data=unicodedata.normalize('NFKD',table.to_html()).encode('ascii','ignore')

,id,title,authors,venue,year
0,304586,The WASA2 object-oriented workflow management ...,"[Gottfried Vossen, Mathias Weske]",International Conference on Management of Data,1999
1,304587,A user-centered interface for querying distrib...,"[Isabel F. Cruz, Kimberly M. James]",International Conference on Management of Data,1999
2,304589,"World Wide Database-integrating the Web, CORBA...","[Athman Bouguettaya, Boualem Benatallah, Lily ...",International Conference on Management of Data,1999
3,304590,XML-based information mediation with MIX,"[Chaitan Baru, Amarnath Gupta, Bertram Lud&#22...",International Conference on Management of Data,1999
4,304582,The CCUBE constraint object-oriented database ...,"[Alexander Brodsky, Victor E. Segal, Jia Chen,...",International Conference on Management of Data,1999
